In [1]:
import numpy as np
import MSMRD as mrd
import MSMRD.integrators as integrators
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [2]:
MFPTS = np.zeros([9,9])                                                                                                                                                                                        
minima = np.array([[-0.9,0.7,0.3] , [-0.1,0.9,0.7],  [0.8,0.8,-0.8],  \
                  [-1.0,-0.3,-0.4], [0.0,0.0,0.0],   [0.9,-0.1,-0.9], \
                  [-0.7,-1.0,-0.3], [0.0,-0.9,0.1],  [0.8,-0.2,0.8]])
class truncTrajsModel3D(object):    
    def __init__(self, entryTrajsStart, entryTrajsEnd, entryTimes, exitTrajs, exitTimes, exitProbs, MSMtime, tmatrix, numPartitions):
        self.entryTrajsStart = entryTrajsStart
        self.entryTrajsEnd = entryTrajsEnd
        self.entryTimes = entryTimes
        self.exitTrajs = exitTrajs
        self.exitTimes = exitTimes
        self.exitProbs = exitProbs
        self.tmatrix = tmatrix
        self.MSMtimestep = MSMtime
        self.numPartitions = numPartitions                                                                                                                                                                         

# Sample random position i sphericall shell between rmin and rmax 
def sampleBathPosition(rmin,rmax):
    outshell = True
    while (outshell):
        rr = np.random.rand()
        rr = rmax*np.cbrt(rr)
        if (rr >= rmin and rr <= rmax):
            outshell = False
    randcosth = 2.0*np.random.rand() - 1.0
    randph = 2.0*np.pi*np.random.rand()
    th = np.arccos(randcosth)
    posx = rr*np.sin(th)*np.cos(randph)
    posy = rr*np.sin(th)*np.sin(randph)
    posz = rr*np.cos(th)
    pos = np.array([posx, posy, posz])
    return pos
                                                                                                                                                                                                               
model = pickle.load(open('../data/models/asym3D/periodicModel_lag10_6files_177partitions.p'))                                                                                                                           
T = np.copy(model.tmatrix) 

In [8]:
r1 = np.zeros(3)                                                                                                                                                                                
p1 = mrd.particle(r1, 1.0)                                                                                                                                                                                 
msm = mrd.MSM(model.tmatrix, minima)                                                                                                                                                                       
msm.exitStates  = []
boundary = mrd.reflectiveSphere(4.)
integrator = integrators.MSMRDtruncTrajs3D(msm, 4.0, p1, 0.01, 2.5, boundary, model)    
integrator.MSM.state = 0
integrator.lastState = 0
integrator.transition = True
integrator.MSMactive = True
integrator.lastStateTime = 0
sim = mrd.simulation(integrator)                                                                                                                                                                           
fpts = [] 
sim.run(100000, sample=True)
traj = sim.traj

In [ ]:
integrator.enterMSM()

In [6]:
traj[0:100]

array([[  0.00000000e+00,   1.00000000e-01,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   2.00000000e-01,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.00000000e+00,   3.00000000e-01,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   4.00000000e+00],
       [  3.00000000e+00,   4.00000000e-01,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   4.00000000e+00],
       [  4.00000000e+00,   5.00000000e-01,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   4.00000000e+00],
       [  5.00000000e+00,   6.00000000e-01,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   4.00000000e+00],
       [  6.00000000e+00,   7.00000000e-01,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   8.00000000e+00],
       [  7.00000000e+00,   8.00000000e-01,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   8.00000000e+00],


In [9]:
print np.where(np.logical_and(traj[1:,5]>=0, traj[:-1,5]==-1))

(array([], dtype=int64),)


In [ ]:
entries = []
#for i in range(9):
entries.append(np.where(np.logical_and(traj[:-1,4]==-1, traj[1:,4] == 0))[0])

In [ ]:
print len(entries[0])

In [ ]:
state = 8
plt.scatter(minima[:,0], minima[:,1])
plt.scatter(minima[state,0], minima[state,1], marker='o')
plt.scatter(traj[entries[state],2], traj[entries[state],3])

In [ ]:
print integrator.exitCalls

In [ ]:
print integrator.entryCalls

In [ ]:
increments = traj[1:,1]-traj[:-1,1]

In [ ]:
sortedTimes = np.argsort(increments)

In [ ]:
plt.plot(increments[sortedTimes], 'x')

In [ ]:
hst = plt.hist(increments, bins = 100)

In [ ]:
print increments[sortedTimes[-1]]

In [ ]:
print traj[sortedTimes[-1]-3:sortedTimes[-1]+2]

In [ ]:
plt.plot(traj[:,0]*0.01, traj[:,1] )

In [ ]:
print len(np.where(traj[:,4] >= 0)[0])

In [ ]:
plt.scatter(traj[:,2], traj[:, 3])

In [ ]:
plt.scatter(traj[sortedTimes[-50:]+1,2], traj[sortedTimes[-50:]+1,3])

In [ ]:
plt.scatter(traj[29319+1,2], traj[29319+1,3])
plt.scatter(minima[:,0], minima[:,1])

In [ ]:
exits = []
for i in range(9):
    exits.append(np.where(np.logical_and(traj[1:,4]==-1, traj[:-1,4] == i))[0]+1)

In [ ]:
state = 0
plt.scatter(minima[:,0], minima[:,1])
plt.scatter(minima[state,0], minima[state,1], marker='o')
plt.scatter(traj[exits[state],2], traj[exits[state],3])

In [ ]:
print len(exits[8])

In [ ]:
print len(exits[8])